# **American Express - Default prediction**

##### *Myroslava Sánchez Andrade A01730712*
##### *Karen Rugerio Armenta A01733228*
##### *José Antonio Bobadilla García A01734433*
##### *Alejandro Castro Reus A01731065*

In [49]:
import pandas as pd
import numpy as np
from dask import dataframe as dd
import matplotlib.pyplot as plt
from sklearn import preprocessing, decomposition, impute
from sklearn.experimental import enable_iterative_imputer

## **Extracting a 20% american express clients sample**

In [31]:
train_data = pd.read_csv('train_data.csv')
train_data = pd.DataFrame(train_data)


In [32]:
amex_clients_results = pd.read_csv("train_labels.csv")

In [33]:
#Shuffle
amex_clients_0 = amex_clients_results[ amex_clients_results["target"] == 0].sample(frac = 1).reset_index(drop = True)
amex_clients_1 = amex_clients_results[ amex_clients_results["target"] == 1].sample(frac = 1).reset_index(drop = True)
amex_clients_0

,customer_ID,target
0,08de336180a9dd9ac8cfecdc7d5cac6f01904c4927167a...,0
1,c0cc82724897bc50515e94e2d39fd546af3cc076d881a1...,0
2,9d4345ffe413a5a2eb2c0b19788c7d23c73de4085a3edf...,0
3,ecfbe4c3a55a370646981b7c94808bc38ee028c192de7b...,0
4,faaca78da7e0766781afd966c30383044f765935a653c5...,0
...,...,...
340080,2348ee34ff83ae7e882c9cb348665e0d6edff608a75211...,0
340081,d0ded21456e902b4d3e10e5631e70662e24702d089d363...,0
340082,b3b41e532e55b491b418b7dc24fb0d47773b02234874de...,0
340083,a6ff0cd63383af88298ecf3d1fa3d20ad88a3ac979fa5e...,0


In [34]:
def get_df_percent(n_clients, percent):
    return round(n_clients * percent)

percent_amex_clients_0 = get_df_percent(amex_clients_0["target"].count(), 0.2)
percent_amex_clients_1 = get_df_percent(amex_clients_1["target"].count(), 0.2)
percent_amex_clients_1

23766

In [35]:
amex_clients_0 = amex_clients_0.head(percent_amex_clients_0)
amex_clients_1 = amex_clients_1.head(percent_amex_clients_1)

In [36]:
amex_clients = pd.concat([amex_clients_1, amex_clients_0], ignore_index = True)
amex_clients

,customer_ID,target
0,8848ea469fbe893a23fc42058e54107754c021c981e795...,1
1,9790f5ca6be4aa4802f8d837d60675e8fdd4de906254a0...,1
2,f3db4fc89abc8e8d59b0d3ef745be808cb70d167252753...,1
3,abd1c241939f3dd8cf1631db407388ad16c9d99993987a...,1
4,53d181bde2da62aebddb59c4f9afb3520fac874e6c11a7...,1
...,...,...
91778,8ce12a4bd1394252e5f9d60ae0ca86c8241571ccc91c82...,0
91779,773302ae250270ab6e756621f0c6b4c1e0bea6cd8c3c6f...,0
91780,60478e3d025e2255395a0430602a3ef428454cf28a9746...,0
91781,88823bba8f995238d4f2c0b29922f7963743a8aaaa9a3c...,0


## **Merging the 20% sample with its correspondent data**

In [37]:
sample_train_data = pd.merge(amex_clients, train_data)

In [38]:
sample_train_data = pd.read_csv("train_labels.csv")

In [39]:
sample_train_data

,customer_ID,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-08-28,0.511769,0.009754,0.181931,0.810619,0.002510,0.177497,0.000863,...,NaN,NaN,NaN,0.000556,0.001755,0.006413,NaN,0.003944,0.007537,0.008857
1,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-09-13,0.507765,0.002712,0.199071,0.819636,0.002847,0.178085,0.003505,...,NaN,NaN,NaN,0.000336,0.004805,0.006488,NaN,0.000357,0.000322,0.003674
2,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-10-15,0.424877,0.000059,0.270903,0.095687,0.004192,0.176206,0.006209,...,NaN,NaN,NaN,0.008027,0.004165,0.002939,NaN,0.003505,0.003345,0.004392
3,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-11-15,0.411393,0.030944,0.346549,0.012620,0.504857,0.291149,0.006342,...,NaN,NaN,NaN,0.007532,0.005044,0.001616,NaN,0.007198,0.008839,0.002233
4,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-12-20,0.336075,0.155478,0.437780,0.013049,0.509996,0.328666,0.009227,...,NaN,NaN,NaN,0.003915,0.005769,0.009580,NaN,0.004344,0.004019,0.006935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107256,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2017-11-07,0.711997,0.326242,0.041635,0.094307,0.005353,-0.004632,0.009840,...,NaN,NaN,NaN,0.002004,0.007816,0.007273,NaN,0.008514,0.001537,0.005309
1107257,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2017-12-29,0.722408,0.037323,0.038988,0.171414,0.000888,0.158201,0.002465,...,NaN,NaN,NaN,0.003526,0.000525,0.004898,NaN,0.007591,0.007101,0.007367
1107258,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2018-01-14,0.687479,0.505748,0.052194,0.173209,0.001695,0.167581,0.009841,...,NaN,NaN,NaN,0.001385,0.006485,0.009664,NaN,0.006995,0.008961,0.005474
1107259,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2018-02-26,0.715083,0.914450,0.062841,0.006116,0.007222,0.171454,0.342117,...,NaN,NaN,NaN,0.004711,0.008107,0.005192,NaN,0.002635,0.003902,0.003772


In [42]:
sample_train_data.to_csv('Clients.csv', index=False)

## **ETL sample (20% of clients)**

In [6]:
sample_train_data = pd.read_csv("clients.csv")

In [7]:
max_val = sample_train_data['customer_ID'].count()
max_val

1107261

In [8]:
sample_train_data = sample_train_data.dropna(thresh = (max_val * 0.70) , axis = 1)
sample_train_data

,customer_ID,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,...,D_130,D_131,D_133,R_28,D_139,D_140,D_141,D_143,D_144,D_145
0,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-08-28,0.511769,0.009754,0.181931,0.810619,0.002510,0.177497,0.000863,...,0.001008,0.009212,0.001381,0.004834,0.000556,0.001755,0.006413,0.003944,0.007537,0.008857
1,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-09-13,0.507765,0.002712,0.199071,0.819636,0.002847,0.178085,0.003505,...,0.006115,0.004586,0.009445,0.004079,0.000336,0.004805,0.006488,0.000357,0.000322,0.003674
2,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-10-15,0.424877,0.000059,0.270903,0.095687,0.004192,0.176206,0.006209,...,0.006432,0.008648,0.009785,0.006931,0.008027,0.004165,0.002939,0.003505,0.003345,0.004392
3,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-11-15,0.411393,0.030944,0.346549,0.012620,0.504857,0.291149,0.006342,...,0.009612,0.009173,0.005924,0.006562,0.007532,0.005044,0.001616,0.007198,0.008839,0.002233
4,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-12-20,0.336075,0.155478,0.437780,0.013049,0.509996,0.328666,0.009227,...,0.002781,0.008375,0.006973,0.001520,0.003915,0.005769,0.009580,0.004344,0.004019,0.006935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107256,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2017-11-07,0.711997,0.326242,0.041635,0.094307,0.005353,-0.004632,0.009840,...,1.007947,0.981901,1.289417,0.001563,0.002004,0.007816,0.007273,0.008514,0.001537,0.005309
1107257,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2017-12-29,0.722408,0.037323,0.038988,0.171414,0.000888,0.158201,0.002465,...,1.000963,0.988021,1.289201,0.008459,0.003526,0.000525,0.004898,0.007591,0.007101,0.007367
1107258,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2018-01-14,0.687479,0.505748,0.052194,0.173209,0.001695,0.167581,0.009841,...,1.000670,0.978812,0.004588,0.001498,0.001385,0.006485,0.009664,0.006995,0.008961,0.005474
1107259,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2018-02-26,0.715083,0.914450,0.062841,0.006116,0.007222,0.171454,0.342117,...,1.004930,0.979756,0.006424,0.000586,0.004711,0.008107,0.005192,0.002635,0.003902,0.003772


In [9]:
cells_per_client = sample_train_data.groupby(sample_train_data.customer_ID).size() * (len(sample_train_data.columns) - 2)
cells_per_client

customer_ID
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    2041
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    2041
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb59b50fec59e0b6381    2041
0001337ded4e1c2539d1a78ff44a457bd4a95caa55ba1730b2849b92ea687f9e     471
0002e335892f7998f0feb3a59f32d652f0da7c85e535b99ea6f87fd317ed47f4    2041
                                                                    ... 
fffb8cb2a4375ee042827ce1e880268ca33b1091bb6811ae06f300fd6514a66b    2041
fffd1a37da07c54a930788ab53061cd7bd34d38592205e4b2bc631c79bcb5160    2041
fffdc3a28b2d17b4136b18ab6a554ccb3d98ec3a9305a0a64255dc7cf8711b40    1099
fffe39594fad367d9eb8ae18537b56b0739c4cc3ab9810f8642d49ec45e60c62    2041
fffeb188719a2fcf32cb4c8d0cc49180b6863eef55dbb9982db73dd460e67f7b    2041
Length: 91783, dtype: int64

In [10]:
non_null_cells_per_client = sample_train_data.groupby(sample_train_data.customer_ID).count()
non_null_cells_per_client = non_null_cells_per_client.drop(columns = ["target"])
non_null_cells_per_client = non_null_cells_per_client.sum(axis = 1)
non_null_cells_per_client

customer_ID
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    2039
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    2002
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb59b50fec59e0b6381    2039
0001337ded4e1c2539d1a78ff44a457bd4a95caa55ba1730b2849b92ea687f9e     405
0002e335892f7998f0feb3a59f32d652f0da7c85e535b99ea6f87fd317ed47f4    1986
                                                                    ... 
fffb8cb2a4375ee042827ce1e880268ca33b1091bb6811ae06f300fd6514a66b    2012
fffd1a37da07c54a930788ab53061cd7bd34d38592205e4b2bc631c79bcb5160    1907
fffdc3a28b2d17b4136b18ab6a554ccb3d98ec3a9305a0a64255dc7cf8711b40    1015
fffe39594fad367d9eb8ae18537b56b0739c4cc3ab9810f8642d49ec45e60c62    2030
fffeb188719a2fcf32cb4c8d0cc49180b6863eef55dbb9982db73dd460e67f7b    2041
Length: 91783, dtype: int64

In [11]:
percentages_non_null_cells_per_clients = (non_null_cells_per_client * 100) / (cells_per_client)
percentages_non_null_cells_per_clients

customer_ID
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5     99.902009
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1     98.089172
0000d17a1447b25a01e42e1ac56b091bb7cbb06317be4cb59b50fec59e0b6381     99.902009
0001337ded4e1c2539d1a78ff44a457bd4a95caa55ba1730b2849b92ea687f9e     85.987261
0002e335892f7998f0feb3a59f32d652f0da7c85e535b99ea6f87fd317ed47f4     97.305243
                                                                       ...    
fffb8cb2a4375ee042827ce1e880268ca33b1091bb6811ae06f300fd6514a66b     98.579128
fffd1a37da07c54a930788ab53061cd7bd34d38592205e4b2bc631c79bcb5160     93.434591
fffdc3a28b2d17b4136b18ab6a554ccb3d98ec3a9305a0a64255dc7cf8711b40     92.356688
fffe39594fad367d9eb8ae18537b56b0739c4cc3ab9810f8642d49ec45e60c62     99.461049
fffeb188719a2fcf32cb4c8d0cc49180b6863eef55dbb9982db73dd460e67f7b    100.000000
Length: 91783, dtype: float64

In [12]:
clients_to_drop = percentages_non_null_cells_per_clients < 25
clients_to_drop = clients_to_drop[clients_to_drop]
clients_to_drop

Series([], dtype: bool)

### *ONE HOT ENCODING*

In [13]:
categorical_columns = sample_train_data[['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_68']]
categorical_columns

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_68
0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,CO,NaN,NaN
1,0.0,1.0,0.0,0.0,-1.0,0.0,1.0,CO,U,3.0
2,0.0,4.0,0.0,0.0,-1.0,0.0,1.0,CO,U,3.0
3,1.0,4.0,0.0,0.0,-1.0,0.0,1.0,CO,U,3.0
4,1.0,4.0,0.0,0.0,-1.0,1.0,1.0,CO,U,3.0
...,...,...,...,...,...,...,...,...,...,...
1107256,0.0,3.0,1.0,0.0,2.0,0.0,-1.0,CO,O,5.0
1107257,0.0,5.0,1.0,0.0,2.0,0.0,0.0,CO,O,5.0
1107258,0.0,3.0,1.0,0.0,2.0,0.0,0.0,CO,O,5.0
1107259,0.0,3.0,1.0,0.0,2.0,0.0,0.0,CO,O,5.0


In [14]:
# Creating a label encoder objecr
le = preprocessing.LabelEncoder()
# Categorizing the column values with number
categorical_columns = categorical_columns.apply(le.fit_transform)
categorical_columns

,B_30,B_38,D_114,D_116,D_117,D_120,D_126,D_63,D_64,D_68
0,0,0,2,2,7,2,3,1,4,7
1,0,0,0,0,0,0,2,1,3,3
2,0,3,0,0,0,0,2,1,3,3
3,1,3,0,0,0,0,2,1,3,3
4,1,3,0,0,0,1,2,1,3,3
...,...,...,...,...,...,...,...,...,...,...
1107256,0,2,1,0,2,0,0,1,1,5
1107257,0,4,1,0,2,0,1,1,1,5
1107258,0,2,1,0,2,0,1,1,1,5
1107259,0,2,1,0,2,0,1,1,1,5


In [15]:
# Creating a one hot enconder object
enc = preprocessing.OneHotEncoder()
enc.fit(categorical_columns)
one_hot_labels = enc.transform(categorical_columns).toarray()
one_hot_labels

array([[1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 1., 0., 0.]])

In [16]:
categorical_column_names = enc.get_feature_names_out(list(categorical_columns.columns))
categorical_column_names

array(['B_30_0', 'B_30_1', 'B_30_2', 'B_30_3', 'B_38_0', 'B_38_1',
       'B_38_2', 'B_38_3', 'B_38_4', 'B_38_5', 'B_38_6', 'B_38_7',
       'D_114_0', 'D_114_1', 'D_114_2', 'D_116_0', 'D_116_1', 'D_116_2',
       'D_117_0', 'D_117_1', 'D_117_2', 'D_117_3', 'D_117_4', 'D_117_5',
       'D_117_6', 'D_117_7', 'D_120_0', 'D_120_1', 'D_120_2', 'D_126_0',
       'D_126_1', 'D_126_2', 'D_126_3', 'D_63_0', 'D_63_1', 'D_63_2',
       'D_63_3', 'D_63_4', 'D_63_5', 'D_64_0', 'D_64_1', 'D_64_2',
       'D_64_3', 'D_64_4', 'D_68_0', 'D_68_1', 'D_68_2', 'D_68_3',
       'D_68_4', 'D_68_5', 'D_68_6', 'D_68_7'], dtype=object)

In [17]:
categorical_columns_df = pd.DataFrame(one_hot_labels, columns = categorical_column_names)
categorical_columns_df

,B_30_0,B_30_1,B_30_2,B_30_3,B_38_0,B_38_1,B_38_2,B_38_3,B_38_4,B_38_5,...,D_64_3,D_64_4,D_68_0,D_68_1,D_68_2,D_68_3,D_68_4,D_68_5,D_68_6,D_68_7
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107256,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107257,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107258,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107259,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [18]:
sample_train_data = sample_train_data.drop(columns = categorical_columns.columns)

In [25]:
sample_train_data = sample_train_data.join(categorical_columns_df)

In [26]:
sample_train_data

,customer_ID,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,...,D_64_3,D_64_4,D_68_0,D_68_1,D_68_2,D_68_3,D_68_4,D_68_5,D_68_6,D_68_7
0,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-08-28,0.511769,0.009754,0.181931,0.810619,0.002510,0.177497,0.000863,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-09-13,0.507765,0.002712,0.199071,0.819636,0.002847,0.178085,0.003505,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-10-15,0.424877,0.000059,0.270903,0.095687,0.004192,0.176206,0.006209,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-11-15,0.411393,0.030944,0.346549,0.012620,0.504857,0.291149,0.006342,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-12-20,0.336075,0.155478,0.437780,0.013049,0.509996,0.328666,0.009227,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107256,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2017-11-07,0.711997,0.326242,0.041635,0.094307,0.005353,-0.004632,0.009840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107257,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2017-12-29,0.722408,0.037323,0.038988,0.171414,0.000888,0.158201,0.002465,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107258,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2018-01-14,0.687479,0.505748,0.052194,0.173209,0.001695,0.167581,0.009841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107259,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2018-02-26,0.715083,0.914450,0.062841,0.006116,0.007222,0.171454,0.342117,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [33]:
sample_train_data.to_csv("customers_one_hot_encoded.csv")

In [ ]:
sample_train_data = pd.read_csv("customers_one_hot_encoded.csv")

In [34]:
sample_train_data

,customer_ID,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,...,D_64_3,D_64_4,D_68_0,D_68_1,D_68_2,D_68_3,D_68_4,D_68_5,D_68_6,D_68_7
0,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-08-28,0.511769,0.009754,0.181931,0.810619,0.002510,0.177497,0.000863,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-09-13,0.507765,0.002712,0.199071,0.819636,0.002847,0.178085,0.003505,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-10-15,0.424877,0.000059,0.270903,0.095687,0.004192,0.176206,0.006209,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-11-15,0.411393,0.030944,0.346549,0.012620,0.504857,0.291149,0.006342,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,8848ea469fbe893a23fc42058e54107754c021c981e795...,1,2017-12-20,0.336075,0.155478,0.437780,0.013049,0.509996,0.328666,0.009227,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107256,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2017-11-07,0.711997,0.326242,0.041635,0.094307,0.005353,-0.004632,0.009840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107257,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2017-12-29,0.722408,0.037323,0.038988,0.171414,0.000888,0.158201,0.002465,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107258,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2018-01-14,0.687479,0.505748,0.052194,0.173209,0.001695,0.167581,0.009841,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1107259,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,0,2018-02-26,0.715083,0.914450,0.062841,0.006116,0.007222,0.171454,0.342117,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [38]:
sample_train_data_x = sample_train_data.drop(columns=['target'])
sample_train_data_y = sample_train_data[['target']]

In [42]:
sample_train_data_x_pca = sample_train_data_x.drop(columns=['customer_ID', 'S_2'])

In [50]:
imp_mean = impute.IterativeImputer()

In [51]:
sample_train_data_x_imputed = imp_mean.fit_transform(sample_train_data_x_pca)
sample_train_data_x_imputed

array([[5.11768556e-01, 9.75371762e-03, 1.81931178e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [5.07764944e-01, 2.71187425e-03, 1.99070585e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.24877008e-01, 5.94825628e-05, 2.70903322e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.87478547e-01, 5.05747969e-01, 5.21936616e-02, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.15082746e-01, 9.14449668e-01, 6.28410004e-02, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.31761538e-01, 5.27642619e-03, 6.38946338e-02, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [54]:
sample_train_data_x_imputed.shape

(1107261, 198)

In [56]:
sample_train_data_x_imputed_df = pd.DataFrame(sample_train_data_x_imputed, columns=list(sample_train_data_x_pca.columns))

In [59]:
sample_train_data_x_imputed.to_csv("customers_imputed_nodate_noid.csv")

In [61]:
sample_train_data_x[['customer_ID', 'S_2']].to_csv("customers_imputed_pca_onlydate_onlyid.csv")

In [62]:
sample_train_data_y.to_csv("customers_imputed_pca_target.csv")

In [80]:
pca = decomposition.PCA(n_components=20) # Ideal: 'mle' -> 184 columnas

In [81]:
sample_train_data_x_pca_imputed_pca_df = pca.fit_transform(sample_train_data_x_pca_imputed_df)

In [82]:
sample_train_data_x_pca_imputed_pca_df = pd.DataFrame(sample_train_data_x_pca_imputed_pca_df, columns=[f'col_{n}' for n in range(sample_train_data_x_pca_imputed_pca_df.shape[1])])
sample_train_data_x_pca_imputed_pca_df

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19
0,-0.277947,0.089819,-0.163377,-0.137094,-0.253049,0.009654,-0.059961,-0.152850,0.335968,-1.704868,-0.002546,-0.063551,0.522591,1.390802,1.867076,-0.715459,0.171778,0.050295,0.712693,1.000121
1,-0.282332,-0.118626,-0.166085,-0.136216,-0.194843,0.023384,-0.212329,-0.143419,0.359913,-1.611096,-0.121155,0.003349,0.044980,0.625777,-0.013568,-0.861939,0.341182,-0.003254,-0.553112,-0.286415
2,-0.273863,-0.119847,-0.174069,-0.089733,0.281766,0.207822,-1.064115,-0.045687,0.379432,-1.293955,-0.144760,-0.017003,-0.062150,-0.811434,-0.000514,0.075977,0.354626,0.290604,-0.026493,-0.209356
3,-0.273336,-0.109205,-0.196930,-0.076145,0.589326,0.232134,-1.890262,-0.143552,0.686512,-1.078072,2.564390,0.147803,0.151988,-1.054538,0.030369,0.202541,0.634660,0.275357,0.025542,-0.573439
4,-0.275455,-0.108884,-0.213906,-0.064950,1.012024,0.435696,-2.456467,-0.192203,0.953909,-1.146337,5.572079,0.375634,0.293019,-0.849146,0.167898,0.163279,0.530454,0.288402,0.168954,-0.575447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107256,-0.279369,-0.111193,-0.165925,-0.067822,0.065406,-0.049359,-0.708944,0.114612,0.024820,0.591585,-0.127586,-0.265176,0.161180,1.208645,0.045287,0.341784,0.638198,0.675008,-0.098012,0.393284
1107257,-0.274746,-0.116872,-0.005602,-0.086628,0.085759,-0.032859,-0.772933,-0.003599,0.121305,0.450044,-0.135509,-0.276362,0.286643,1.151254,0.251762,0.374166,0.842693,0.723328,0.074701,0.103220
1107258,-0.275069,-0.119606,-0.006950,-0.072468,0.094371,-0.035675,-0.811284,0.111312,0.067754,0.509525,-0.131702,-0.256507,0.161593,1.091158,0.103654,0.361997,0.823487,0.738349,0.039509,0.461184
1107259,-0.271679,-0.114800,-0.004740,0.076209,0.103751,-0.047498,-0.791178,0.105125,0.025009,0.487947,-0.127133,-0.227164,0.117920,1.119868,0.022523,0.311845,0.598651,0.873023,0.100629,0.656888


In [83]:
sample_train_data_pca = sample_train_data_y.join(sample_train_data_x[['customer_ID', 'S_2']]).join(sample_train_data_x_pca_imputed_pca_df)
sample_train_data_pca

,target,customer_ID,S_2,col_0,col_1,col_2,col_3,col_4,col_5,col_6,...,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19
0,1,8848ea469fbe893a23fc42058e54107754c021c981e795...,2017-08-28,-0.277947,0.089819,-0.163377,-0.137094,-0.253049,0.009654,-0.059961,...,-0.002546,-0.063551,0.522591,1.390802,1.867076,-0.715459,0.171778,0.050295,0.712693,1.000121
1,1,8848ea469fbe893a23fc42058e54107754c021c981e795...,2017-09-13,-0.282332,-0.118626,-0.166085,-0.136216,-0.194843,0.023384,-0.212329,...,-0.121155,0.003349,0.044980,0.625777,-0.013568,-0.861939,0.341182,-0.003254,-0.553112,-0.286415
2,1,8848ea469fbe893a23fc42058e54107754c021c981e795...,2017-10-15,-0.273863,-0.119847,-0.174069,-0.089733,0.281766,0.207822,-1.064115,...,-0.144760,-0.017003,-0.062150,-0.811434,-0.000514,0.075977,0.354626,0.290604,-0.026493,-0.209356
3,1,8848ea469fbe893a23fc42058e54107754c021c981e795...,2017-11-15,-0.273336,-0.109205,-0.196930,-0.076145,0.589326,0.232134,-1.890262,...,2.564390,0.147803,0.151988,-1.054538,0.030369,0.202541,0.634660,0.275357,0.025542,-0.573439
4,1,8848ea469fbe893a23fc42058e54107754c021c981e795...,2017-12-20,-0.275455,-0.108884,-0.213906,-0.064950,1.012024,0.435696,-2.456467,...,5.572079,0.375634,0.293019,-0.849146,0.167898,0.163279,0.530454,0.288402,0.168954,-0.575447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107256,0,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,2017-11-07,-0.279369,-0.111193,-0.165925,-0.067822,0.065406,-0.049359,-0.708944,...,-0.127586,-0.265176,0.161180,1.208645,0.045287,0.341784,0.638198,0.675008,-0.098012,0.393284
1107257,0,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,2017-12-29,-0.274746,-0.116872,-0.005602,-0.086628,0.085759,-0.032859,-0.772933,...,-0.135509,-0.276362,0.286643,1.151254,0.251762,0.374166,0.842693,0.723328,0.074701,0.103220
1107258,0,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,2018-01-14,-0.275069,-0.119606,-0.006950,-0.072468,0.094371,-0.035675,-0.811284,...,-0.131702,-0.256507,0.161593,1.091158,0.103654,0.361997,0.823487,0.738349,0.039509,0.461184
1107259,0,6d0e6b89799a18495cdee27d5cbf7b0df2c95c98ed9ed1...,2018-02-26,-0.271679,-0.114800,-0.004740,0.076209,0.103751,-0.047498,-0.791178,...,-0.127133,-0.227164,0.117920,1.119868,0.022523,0.311845,0.598651,0.873023,0.100629,0.656888


In [84]:
sample_train_data_pca.to_csv("customers_pca_20.csv")